In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
    library(DescTools)
}))


In [2]:
source('paths.sh')
source(snpnet_helper)
source(fPRS_helper)


In [3]:
args <- c(
    'dev.BIN4093',
    phe_f,
    'BIN4093',
    'binomial',
    'age,sex,Array,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10',
    'train_val=train_val,test=train_val,non_british_white=non_british_white,african=african,s_asian=s_asian,e_asian=e_asian',
    file.path(project_d, PRS_f)
)


In [4]:
# pase command line args
# output
performance_eval_prefix <- args[1] # the output file containing the predictive performance
## inputs
pheno_and_covar_f <- args[2] # the master phenotype file with covariates
pheno_col <- args[3] # the phenotype column
family <- args[4] # gaussian or binomial
covariates_str <- args[5]
split_strs <- args[6] # list of split groups to consider
sscore_f <- args[7] # the sscore input file


In [5]:
####################################################################
# main

## parse lists

covariates_str %>% split_list_str() -> covariates

split_strs %>% split_named_list_str() -> population_splits

# set gaussian and binomial phenotype lists
stopifnot(family %in% c('gaussian', 'binomial'))
if(family == 'gaussian'){
    phes_binary <- NULL
    phes_quantitative <- c(pheno_col)
}else if(family == 'binomial'){
    phes_binary <- c(pheno_col)
    phes_quantitative <- NULL
}

# regression formula
covar_formula_str <- sprintf(
    '%s ~ 1 + %s',
    pheno_col, paste(covariates, collapse=' + ')
)

# list of risk score models we consider in the evaluation
score_geno  <- paste0('PRS_', pheno_col)
score_covar <- paste0('covar_', pheno_col)
score_full  <- paste0('full_', pheno_col)


In [6]:
# read phenotype
pheno_and_covar_f %>%
read_phenotype_file(c(covariates, phes_binary, phes_quantitative)) %>%
recode_pheno_values(phes_binary, phes_quantitative) %>%
update_split_column_for_refit() -> pheno_df


In [7]:
pheno_df %>%
filter(split %in% names(population_splits)) %>%
count(Array, population)


Array,population,n
<int>,<chr>,<int>
0,non_british_white,2499
0,white_british,37035
1,african,6497
1,e_asian,1704
1,non_british_white,22406
1,s_asian,7831
1,white_british,300094


In [8]:
# we fit the specified regression model for each split independently
# and aggregate the results into one data frame
population_splits %>% unique() %>%
lapply(function(s){
    pheno_df %>%
    filter(split == s) %>%
    fit_glm(covar_formula_str, family) %>%
    fit_to_df() %>%
    mutate(split = s) %>%
    select(split, variable, estimate, SE, z_or_t_value, P)
}) %>%
bind_rows() -> covar_model_BETAs_df


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: algorithm did not converge”


In [9]:
covar_model_BETAs_df %>%
rename('#split' = 'split') %>%
fwrite(sprintf('%s.covarBETAs.tsv', performance_eval_prefix), sep='\t', na = "NA", quote=F)


In [10]:
# we fit the specified regression model for each split independently
# and aggregate the results into one data frame
population_splits %>% names() %>%
lapply(function(s){
    # use the BETAs on a split specified in named list, PRS_model_covar_BETAs_split
    covar_score_split <- (population_splits[[s]])
    
    # get BETAs
    covar_model_BETAs_df %>%
    filter(split == covar_score_split) %>%
    rename(!!score_covar := 'estimate') -> covar_betas_pop_df

    # loop across different split
    pheno_df %>%
    filter(split == s) %>%
    FID_IID_to_rownames() %>%
    compute_matrix_product(
        covar_betas_pop_df,
        covar_betas_pop_df %>% pull(variable) %>% intersect(covariates),
        beta_estimate_cols=c(score_covar)
    ) %>%
    mutate(covar_score_computed_on = covar_score_split) %>%
    FID_IID_from_rownames()
}) %>%
bind_rows() -> covar_score_df


In [11]:
# read sscore file
message(sprintf('.. reading %s', sscore_f))
sscore_f %>%
read_sscore_file(columns = score_geno) -> sscore_df


.. reading /scratch/groups/mrivas/projects/PRS/private_output/202009_batch/ukb24983_GWAS_covar.20200828.PRSs.phe.gz



In [12]:
# join all the individual-level data into one data frame
pheno_df %>%
filter(split %in% names(population_splits)) %>%
left_join(sscore_df, by=c('FID', 'IID')) %>%
left_join(covar_score_df, by=c("FID", "IID")) %>%
mutate(!!score_full := rowSums(across(all_of(c(score_covar))))) -> full_df


In [13]:
# regression formula
covarPRS_formula_str <- sprintf(
    '%s ~ 1 + (1*%s) + %s',
    pheno_col, score_covar, score_geno
)

# we fit the specified regression model for each split independently
# and aggregate the results into one data frame
population_splits %>% names() %>%
lapply(function(s){
    full_df %>%
    filter(split == s) %>%
    fit_glm(covarPRS_formula_str, family) %>%
    fit_to_df() %>%
    mutate(split = s) %>%
    select(split, variable, estimate, SE, z_or_t_value, P)
}) %>%
bind_rows() -> covarPRS_model_BETAs_df


Warning message:
“glm.fit: algorithm did not converge”


In [14]:
covarPRS_model_BETAs_df

split,variable,estimate,SE,z_or_t_value,P
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
train_val,(Intercept),-6.054701e+00,6.594923e-02,-9.180852e+01,0.000000e+00
train_val,PRS_BIN4093,5.828249e+00,1.189613e+00,4.899281e+00,9.618814e-07
test,(Intercept),-6.061036e+00,1.283013e-01,-4.724065e+01,0.000000e+00
test,PRS_BIN4093,-1.328285e+00,2.546403e+00,-5.216320e-01,6.019266e-01
non_british_white,(Intercept),-6.225325e+00,2.209528e-01,-2.817491e+01,1.187288e-174
non_british_white,PRS_BIN4093,-7.453305e+00,4.446692e+00,-1.676146e+00,9.370960e-02
african,(Intercept),-5.904492e+00,5.855914e-01,-1.008296e+01,6.571919e-24
african,PRS_BIN4093,-2.935046e+00,7.472681e+00,-3.927701e-01,6.944893e-01
s_asian,(Intercept),-6.910233e+00,5.139545e-01,-1.344522e+01,3.284142e-41


In [18]:
# list of "scores" we will use in the evaluation
c(score_geno, score_covar, score_full) -> risk_score_list


In [22]:
full_df %>% count_n_per_split(pheno_col, family) -> split_cnt_df
if (family == 'binomial') {
    # run evaluation
    split_cnt_df %>%
    filter(case_n > 0, control_n > 0) %>%
    pull(split) -> non_zero_splits
} else {
    split_cnt_df %>%
    filter(n > 0) %>%
    pull(split) -> non_zero_splits    
}

In [28]:
# run evaluation
non_zero_splits  %>% 
intersect(names(population_splits)) %>%
lapply(function(split_str){
    risk_score_list %>% lapply(function(predictor){
        message(sprintf('--%s %s', split_str, predictor))
        tryCatch({
            full_df %>% filter(split == split_str) -> filtered_df
            if(length(filtered_df %>% pull(all_of(pheno_col)) %>% unique())>1){
                filtered_df %>%
                eval_CI(pheno_col, c(all_of(predictor)), family) %>%
                mutate(split = split_str)
            }else{
                message(sprintf(' .. skip (the phenotype value is constant in %s', split_str))
            }
        }, error=function(e){print(e)})
    }) %>% bind_rows()
}) %>%
bind_rows() %>%
left_join(
    full_df %>% count_n_per_split(pheno_col, family),
    by = "split"
) -> PRS_eval_df


--train_val PRS_BIN4093

--train_val covar_BIN4093

--train_val full_BIN4093

--test PRS_BIN4093

--test covar_BIN4093

--test full_BIN4093

--non_british_white PRS_BIN4093

--non_british_white covar_BIN4093

--non_british_white full_BIN4093

--african PRS_BIN4093

--african covar_BIN4093

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
--african full_BIN4093

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
--s_asian PRS_BIN4093

--s_asian covar_BIN4093

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
--s_asian full_BIN4093

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurre

In [32]:
PRS_eval_df %>%
rename('#response' = 'response') %>%
fwrite(sprintf('%s.eval.tsv', performance_eval_prefix), sep='\t', na = "NA", quote=F)


In [33]:
# prepare data frames for the plots
full_df %>%
filter(split == 'test') %>%
drop_na(all_of(c(score_geno, pheno_col))) %>%
rename('geno_score' := all_of(score_geno)) %>%
rename('phe' := all_of(pheno_col)) %>%
mutate(
    phe = phe + 1,
    geno_score_percentile = rank(-geno_score) / n()
) -> plot_df

In [34]:
summary_plot_df <- plot_df %>%
compute_summary_df('geno_score_percentile', 'phe', family=family)


Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(percentile_col)` instead of `percentile_col` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [35]:
summary_plot_df %>%
rename('#l_bin' = 'l_bin') %>%
fwrite(sprintf('%s.percentile.tsv', performance_eval_prefix), sep='\t', na = "NA", quote=F)


In [36]:

if(family == 'gaussian'){
    p1 <- plot_df %>% plot_PRS_vs_phe() +
    theme(legend.position=c(.1, .8))+
    labs(title = pheno_col, y = pheno_col)

    p2 <- summary_plot_df %>%
    plot_PRS_bin_vs_phe(mean(plot_df$phe))+
    labs(title = pheno_col, y = pheno_col)
}else if(family == 'binomial'){
    p1 <- plot_df %>% plot_PRS_binomial() +
    labs(title = pheno_col, x = pheno_col)

    p2 <- summary_plot_df %>% plot_PRS_bin_vs_OR() +
    labs(title = pheno_col)
}else{
    stop(sprintf('%s family is not supported!', family))
}

for(ext in c('png', 'pdf')){ggsave(
    sprintf('%s.plot.%s', performance_eval_prefix, ext),
    gridExtra::arrangeGrob(p1, p2, ncol=2),
    width=12, height=6
)}
